**DATASET CLASS**


In [6]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


Loading in Our Data

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset

class HARWindowDataset(Dataset): 
    def __init__(self, data, labels, window_size=128, stride=64): 
        self.X = [] 
        self.y = [] 
 
        for i in range(0, len(data) - window_size, stride): 
            window = data[i:i + window_size] 
            label = labels[i:i + window_size] 
            majority = np.bincount(label).argmax() 
 
            self.X.append(window) 
            self.y.append(majority) 
 
        self.X = torch.tensor(self.X, dtype=torch.float32) 
        self.y = torch.tensor(self.y, dtype=torch.long) 
 
    def __len__(self): 
        return len(self.X) 
 
    def __getitem__(self, idx): 
        return self.X[idx], self.y[idx] 
 

In [8]:
class ActivityModel(nn.Module): 
    def __init__(self, input_channels=6, num_classes=6): 
        super(ActivityModel, self).__init__() 
        self.cnn = nn.Sequential( 
            nn.Conv1d(input_channels, 32, kernel_size=5), 
            nn.ReLU(), 
            nn.MaxPool1d(2) 
        ) 
        self.lstm = nn.LSTM(input_size=32, hidden_size=64, batch_first=True) 
        self.classifier = nn.Sequential( 
            nn.Linear(64, 32), 
            nn.ReLU(), 
            nn.Linear(32, num_classes) 
        ) 
 
    def forward(self, x):  # x shape: [B, T, C] 
        x = x.permute(0, 2, 1)  # to [B, C, T] 
        x = self.cnn(x)         # [B, 32, T'] 
        x = x.permute(0, 2, 1)  # [B, T', 32] 
        _, (h_n, _) = self.lstm(x) 
        return self.classifier(h_n.squeeze(0))

In [ ]:
model = ActivityModel().to('cuda') 
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) 
 
for epoch in range(10): 
    model.train() 
    total_loss = 0 
    for x_batch, y_batch in train_loader: 
        x_batch, y_batch = x_batch.to('cuda'), y_batch.to('cuda') 
        optimizer.zero_grad() 
        output = model(x_batch) 
        loss = criterion(output, y_batch) 
        loss.backward() 
        optimizer.step() 
        total_loss += loss.item() 
     
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}") 
 